### Downstream task

In [136]:
from pathlib import Path
import time
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import numpy as np

In [111]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding=0)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv_block1 = ConvBlock(1, 32, 8)
        self.conv_block2 = ConvBlock(32, 64, 4)
        self.conv_block3 = ConvBlock(64, 128, 2)
        
        self.max_pooling = nn.MaxPool1d(kernel_size=4, stride=2)
        self.global_max_pooling = nn.AdaptiveMaxPool1d(1)
        
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.max_pooling(x)
        
        x = self.conv_block2(x)
        x = self.max_pooling(x)

        x = self.conv_block3(x)
        x = self.global_max_pooling(x)
        
        x = self.flatten(x)

        return x

class DownstreamEncoder(nn.Module):
    def __init__(self):
        super(DownstreamEncoder, self).__init__()
        self.encoder = Encoder()
    
    def forward(self, x):
        x = self.encoder(x)

        return x

In [112]:
premodel_state_paths = sorted([sensor for sensor in Path('pretrained_models').iterdir()], key=lambda s: s.stem)

premodel_states = []
for premodel_state_path in premodel_state_paths:
    premodel_state = torch.load(premodel_state_path)

    layers_to_remove = [name for name in premodel_state.keys() if 'projection' in name]
    
    for name in layers_to_remove:
        del premodel_state[name]
    
    premodel_states.append(premodel_state)

In [113]:
encoders = [DownstreamEncoder()] * len(premodel_states)
for i,encoder in enumerate(encoders):
    encoder.load_state_dict(premodel_states[i])
    for param in encoder.parameters():
        param.requires_grad = False
    encoder = encoder.cuda()

In [114]:
print(len(encoders))

2


In [120]:
class DownstreamModel(nn.Module):
    def __init__(self, embed_dim, num_heads, num_sensors) -> None:
        super(DownstreamModel, self).__init__()

        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.classifier = nn.Linear(num_sensors*embed_dim, 1)
    
    def forward(self, features):
        x, _ = self.attention(features, features, features)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x

In [130]:
model = DownstreamModel(embed_dim=128, num_heads=8, num_sensors=2)
model = model.cuda()

### Dataset

In [123]:
class MultimodalDataset(Dataset):
    def __init__(self, path:Path, sensor_names:list[str]):
        self.path = path
        self.sensor_name = sensor_names
        self.n_sensors = len(sensor_names)
        df = pd.read_csv(self.path, index_col='timestamp')
        self.input_sequences = df.loc[:,sensor_names]
        self.labels = df.loc[:, 'label']


    def __getitem__(self, index) -> tuple[torch.Tensor, float]:
        sequence_index = self.input_sequences.index.unique()
        original_sequence = self.input_sequences.loc[sequence_index[index]]
        label = self.labels.loc[sequence_index[index]]
        return torch.Tensor(original_sequence.to_numpy().reshape(self.n_sensors, -1)), label.iloc[0]


    def __len__(self):
        return len(self.input_sequences.index.unique())

In [124]:
batch_size = 128
sensors = sorted(['AML','EDA'])
dataset = MultimodalDataset(Path('Intermediate/proc/labeled_joined/P01_labeled.csv'), sensors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [142]:
participant_labeled_data = [labeled_path for labeled_path in Path('Intermediate/proc/labeled_joined').iterdir()]
participants_num = len(participant_labeled_data)
random.shuffle(participant_labeled_data)

train_datasets = [MultimodalDataset(path, sensors) for path in participant_labeled_data[:int(0.8*participants_num)]]
train_dataloader = [DataLoader(dataset, batch_size, shuffle=True) for dataset in train_datasets]

val_datasets = [MultimodalDataset(path, sensors) for path in participant_labeled_data[int(0.8*participants_num):int(0.9*participants_num)]]
val_dataloaders = [DataLoader(dataset, batch_size, shuffle=False) for dataset in val_datasets]

test_datasets = [MultimodalDataset(path, sensors) for path in participant_labeled_data[int(0.9*participants_num):]]
test_dataloaders = [DataLoader(dataset, batch_size, shuffle=False) for dataset in test_datasets]

In [145]:
tr_ep_loss = []
tr_ep_acc = []

val_ep_loss = []
val_ep_acc = []

min_val_loss = 100.0

EPOCHS = 10

dsoptimizer = torch.optim.SGD(model.parameters(),lr = 0.01, momentum = 0.9)

lr_scheduler = torch.optim.lr_scheduler.StepLR(dsoptimizer, step_size=1, gamma=0.98, last_epoch=-1, verbose = True)

loss_fn = nn.BCEWithLogitsLoss()

Adjusting learning rate of group 0 to 1.0000e-02.


In [146]:
for epoch in range(EPOCHS):
    
    stime = time.time()
    print("=============== Epoch : %3d ==============="%(epoch+1))
    
    loss_sublist = np.array([])
    acc_sublist = np.array([])
    
    #iter_num = 0
    model.train()
    
    dsoptimizer.zero_grad()
    
    for dataloader in train_dataloader:
        for x,y in dataloader:
            x = x.cuda()
            y = y.unsqueeze(1).cuda()
            
            features = [encoder(x[:,i,:].unsqueeze(1)).unsqueeze(1) for i,encoder in enumerate(encoders)]
        
            features = torch.concat(features, dim=1)

            z = model(features)
            
            dsoptimizer.zero_grad()
            
            tr_loss = loss_fn(z,y)
            tr_loss.backward()

            preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))
            
            dsoptimizer.step()
            
            loss_sublist = np.append(loss_sublist, tr_loss.cpu().data)
            acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
        
    print('ESTIMATING TRAINING METRICS.............')
    
    print('TRAINING BINARY CROSSENTROPY LOSS: ',np.mean(loss_sublist))
    print('TRAINING BINARY ACCURACY: ',np.mean(acc_sublist))
    
    tr_ep_loss.append(np.mean(loss_sublist))
    tr_ep_acc.append(np.mean(acc_sublist))
    
    print('ESTIMATING VALIDATION METRICS.............')
    
    model.eval()
    
    loss_sublist = np.array([])
    acc_sublist = np.array([])
    
    with torch.no_grad():
        for dataloader in val_dataloaders:
            for x,y in dataloader:
                x = x.cuda()
                y = y.unsqueeze(1).cuda()
                
                features = [encoder(x[:,i,:].unsqueeze(1)).unsqueeze(1) for i,encoder in enumerate(encoders)]
        
                features = torch.concat(features, dim=1)

                z = model(features)

                val_loss = loss_fn(z,y)

                preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))

                loss_sublist = np.append(loss_sublist, val_loss.cpu().data)
                acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
    
    print('VALIDATION BINARY CROSSENTROPY LOSS: ',np.mean(loss_sublist))
    print('VALIDATION BINARY ACCURACY: ',np.mean(acc_sublist))
    
    val_ep_loss.append(np.mean(loss_sublist))
    val_ep_acc.append(np.mean(acc_sublist))
    
    lr_scheduler.step()
    
    if np.mean(loss_sublist) <= min_val_loss:
        min_val_loss = np.mean(loss_sublist) 
        print('Saving model...')
        torch.save(model.state_dict(), 'downstream_models/downstream.pt')
    
    print("Time Taken : %.2f minutes"%((time.time()-stime)/60.0))

=============== Epoch :   1 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.6435069840800101
TRAINING BINARY ACCURACY:  0.6691887347625053
ESTIMATING VALIDATION METRICS.............
VALIDATION BINARY CROSSENTROPY LOSS:  0.5759311267930418
VALIDATION BINARY ACCURACY:  0.7332480409403487
Adjusting learning rate of group 0 to 9.8000e-03.
Saving model...
Time Taken : 6.05 minutes
=============== Epoch :   2 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.64512957050134
TRAINING BINARY ACCURACY:  0.6691887347625053
ESTIMATING VALIDATION METRICS.............
VALIDATION BINARY CROSSENTROPY LOSS:  0.5760034060621088
VALIDATION BINARY ACCURACY:  0.7332480409403487
Adjusting learning rate of group 0 to 9.6040e-03.
Time Taken : 6.00 minutes
=============== Epoch :   3 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.6440213805947722
TRAINING BINARY ACCURACY: 

KeyboardInterrupt: 

In [149]:
model = DownstreamModel(128, 8 ,2).cuda()
model.load_state_dict(torch.load('downstream_models/downstream.pt'))

model.eval()
    
loss_sublist = np.array([])
acc_sublist = np.array([])

with torch.no_grad():
    for dataloader in test_dataloaders:
        for x,y in dataloader:
            x = x.cuda()
            y = y.unsqueeze(1).cuda()
            
            features = [encoder(x[:,i,:].unsqueeze(1)).unsqueeze(1) for i,encoder in enumerate(encoders)]
        
            features = torch.concat(features, dim=1)

            z = model(features)

            val_loss = loss_fn(z,y)

            preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))

            loss_sublist = np.append(loss_sublist, val_loss.cpu().data)
            acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)

print('TEST BINARY CROSSENTROPY LOSS: ',np.mean(loss_sublist))
print('TEST BINARY ACCURACY: ',np.mean(acc_sublist))

TEST BINARY CROSSENTROPY LOSS:  0.7747196915014206
TEST BINARY ACCURACY:  0.5250884622077847
